In [79]:
import pandas as pd
import numpy as np
import os
import random
import re

In [80]:
file_list = pd.read_csv("/Users/ktruong/data/AllTheBacteria/file_list.all.latest.tsv", sep="\t")

In [81]:
print("length:", file_list.shape[0])
file_list.head()

length: 2440377


,sample,species_sylph,species_miniphy,filename_in_tar_xz,tar_xz,tar_xz_url,tar_xz_md5,tar_xz_size_MB
0,SAMN12335635,Achromobacter xylosoxidans,achromobacter_xylosoxidans,atb.assembly.r0.2.batch.1/SAMN12335635.fa,atb.assembly.r0.2.batch.1.tar.xz,https://osf.io/download/667142936b6c8e33f404cce7/,65973bf642ddd13bfd4ffffc37b72efb,32.67
1,SAMN12335634,Achromobacter xylosoxidans,achromobacter_xylosoxidans,atb.assembly.r0.2.batch.1/SAMN12335634.fa,atb.assembly.r0.2.batch.1.tar.xz,https://osf.io/download/667142936b6c8e33f404cce7/,65973bf642ddd13bfd4ffffc37b72efb,32.67
2,SAMN12335574,Achromobacter xylosoxidans,achromobacter_xylosoxidans,atb.assembly.r0.2.batch.1/SAMN12335574.fa,atb.assembly.r0.2.batch.1.tar.xz,https://osf.io/download/667142936b6c8e33f404cce7/,65973bf642ddd13bfd4ffffc37b72efb,32.67
3,SAMN12335579,Achromobacter xylosoxidans,achromobacter_xylosoxidans,atb.assembly.r0.2.batch.1/SAMN12335579.fa,atb.assembly.r0.2.batch.1.tar.xz,https://osf.io/download/667142936b6c8e33f404cce7/,65973bf642ddd13bfd4ffffc37b72efb,32.67
4,SAMN23768638,Achromobacter xylosoxidans,achromobacter_xylosoxidans,atb.assembly.r0.2.batch.1/SAMN23768638.fa,atb.assembly.r0.2.batch.1.tar.xz,https://osf.io/download/667142936b6c8e33f404cce7/,65973bf642ddd13bfd4ffffc37b72efb,32.67


In [82]:
batch_sizes = file_list.groupby(['tar_xz','species_miniphy']).groups

In [83]:
for key in batch_sizes.keys():
    print(key[0],',',key[1],',',len(batch_sizes[key]))

atb.assembly.incr_release.202408.batch.1.tar.xz , acinetobacter_baumannii , 4000
atb.assembly.incr_release.202408.batch.10.tar.xz , bacteroides_thetaiotaomicron , 354
atb.assembly.incr_release.202408.batch.100.tar.xz , klebsiella_pneumoniae , 3886
atb.assembly.incr_release.202408.batch.101.tar.xz , klebsiella_quasipneumoniae , 623
atb.assembly.incr_release.202408.batch.102.tar.xz , klebsiella_variicola , 571
atb.assembly.incr_release.202408.batch.103.tar.xz , lacticaseibacillus_paracasei , 242
atb.assembly.incr_release.202408.batch.104.tar.xz , lacticaseibacillus_rhamnosus , 280
atb.assembly.incr_release.202408.batch.105.tar.xz , legionella_fraseri , 252
atb.assembly.incr_release.202408.batch.106.tar.xz , legionella_pneumophila , 1827
atb.assembly.incr_release.202408.batch.107.tar.xz , limosilactobacillus_fermentum , 267
atb.assembly.incr_release.202408.batch.108.tar.xz , listeria_innocua , 222
atb.assembly.incr_release.202408.batch.109.tar.xz , listeria_monocytogenes , 3339
atb.assemb

In [84]:
species_sizes = file_list.groupby(['species_miniphy','tar_xz']).size()
type(species_sizes)

pandas.core.series.Series

In [85]:
species_sizes.sort_values(ascending=True)

species_miniphy                  tar_xz                                           
1140203949_sp002787635           atb.assembly.r0.2.batch.92.tar.xz                       1
novosphingopyxis_baekryungensis  atb.assembly.r0.2.batch.125.tar.xz                      1
nsj61_sp003433845                atb.assembly.incr_release.202408.batch.48.tar.xz        1
occultella_aeris                 atb.assembly.incr_release.202408.batch.48.tar.xz        1
oceanibacterium_hippocampi       atb.assembly.r0.2.batch.125.tar.xz                      1
                                                                                      ... 
salmonella_enterica              atb.assembly.r0.2.batch.412.tar.xz                   4000
                                 atb.assembly.r0.2.batch.413.tar.xz                   4000
                                 atb.assembly.r0.2.batch.414.tar.xz                   4000
                                 atb.assembly.incr_release.202408.batch.177.tar.xz    4000
strepto

In [86]:
def extract_prefix_and_number(s):
    match = re.match(r"([A-Z]+)(\d+)", s)
    if not match:
        return None, None
    return match.group(1), int(match.group(2))

def group_accessions_by_range(accessions):
    accessions = sorted(accessions, key=lambda x: extract_prefix_and_number(x))
    grouped = []

    i = 0
    while i < len(accessions):
        start = accessions[i]
        prefix, num = extract_prefix_and_number(start)
        j = i
        while j + 1 < len(accessions):
            next_prefix, next_num = extract_prefix_and_number(accessions[j + 1])
            if next_prefix == prefix and next_num == num + 1:
                j += 1
                num += 1
            else:
                break
        end = accessions[j]
        length = j - i + 1
        if i == j:
            grouped.append(([start], 1))
        else:
            grouped.append(([start, end], length))
        i = j + 1
    return grouped

# Example input
accessions = [
    "SAMD00000344", "SAMD00000345", "SAMD00000355", "SAMD00000550", 
    "SAMD00000552", "SAMN10407504", "SAMN10407505", "SAMN10407506"
]

# Run grouping
groups = group_accessions_by_range(accessions)

# Print results
for group in groups:
    print(group)

(['SAMD00000344', 'SAMD00000345'], 2)
(['SAMD00000355'], 1)
(['SAMD00000550'], 1)
(['SAMD00000552'], 1)
(['SAMN10407504', 'SAMN10407506'], 3)


In [87]:
groups_accession = group_accessions_by_range(file_list['sample'])

In [88]:
groups_accession

[(['SAMD00000344', 'SAMD00000345'], 2),
 (['SAMD00000355'], 1),
 (['SAMD00000550'], 1),
 (['SAMD00000552'], 1),
 (['SAMD00000562'], 1),
 (['SAMD00000572'], 1),
 (['SAMD00000584', 'SAMD00000585'], 2),
 (['SAMD00000605'], 1),
 (['SAMD00000611', 'SAMD00000612'], 2),
 (['SAMD00000644'], 1),
 (['SAMD00000649'], 1),
 (['SAMD00000652'], 1),
 (['SAMD00000665', 'SAMD00000667'], 3),
 (['SAMD00000669'], 1),
 (['SAMD00000692'], 1),
 (['SAMD00000699'], 1),
 (['SAMD00000724'], 1),
 (['SAMD00000754'], 1),
 (['SAMD00000756', 'SAMD00000760'], 5),
 (['SAMD00000768'], 1),
 (['SAMD00000798'], 1),
 (['SAMD00000806'], 1),
 (['SAMD00000812'], 1),
 (['SAMD00000816'], 1),
 (['SAMD00002586', 'SAMD00002593'], 8),
 (['SAMD00002642'], 1),
 (['SAMD00002680', 'SAMD00002681'], 2),
 (['SAMD00002684'], 1),
 (['SAMD00002686'], 1),
 (['SAMD00002690', 'SAMD00002691'], 2),
 (['SAMD00002693'], 1),
 (['SAMD00002695'], 1),
 (['SAMD00002788', 'SAMD00002789'], 2),
 (['SAMD00002791'], 1),
 (['SAMD00002802'], 1),
 (['SAMD00002817

In [89]:
# Convert to DataFrame
rows = []
for group, length in groups_accession:
    if len(group) == 1:
        start = end = group[0]
    else:
        start, end = group
    rows.append({"start": start, "end": end, "length": length})

df_range = pd.DataFrame(rows)
print(df_range)

               start           end  length
0       SAMD00000344  SAMD00000345       2
1       SAMD00000355  SAMD00000355       1
2       SAMD00000550  SAMD00000550       1
3       SAMD00000552  SAMD00000552       1
4       SAMD00000562  SAMD00000562       1
...              ...           ...     ...
233877  SAMN42785595  SAMN42785595       1
233878  SAMN42786914  SAMN42786914       1
233879  SAMN42786917  SAMN42786917       1
233880  SAMN42786919  SAMN42786919       1
233881  SAMN42786928  SAMN42786928       1

[233882 rows x 3 columns]


In [90]:
def check_species_consistency(groups, original_df):
    """
    Check if all accessions in each group belong to the same species.

    Args:
        groups (list): List of tuples like ([start, end], length).
        original_df (pd.DataFrame): DataFrame with 'accession' and 'species' columns.

    Returns:
        list of dicts: Each dict includes start, end, length, consistent (bool), and species (if consistent).
    """
    result = []
    
    # Set index to accession for fast lookup
    df = original_df.set_index('sample')
    total = len(groups)
    current=1
    for group, length in groups:
        print(current,'/',total)
        current+=1
        # Expand group: from [start, end] get all accessions between them (inclusive)
        if len(group) == 1:
            accessions = [group[0]]
        else:
            start, end = group
            # Get all accession names in order from the original dataframe
            all_accessions = df.index.tolist()
            try:
                i = all_accessions.index(start)
                j = all_accessions.index(end)
                if i > j:
                    i, j = j, i
                accessions = all_accessions[i:j+1]
            except ValueError:
                accessions = group  # fallback if not found

        species_in_group = df.loc[accessions]['species_sylph'].unique()
        if len(species_in_group) == 1:
            consistent = True
            species = species_in_group[0]
        else:
            consistent = False
            species = None

        result.append({
            "start": group[0],
            "end": group[-1],
            "length": length,
            "consistent": consistent,
            "species": species
        })

    return pd.DataFrame(result)

In [91]:
# df_consistency = check_species_consistency(groups_accession,file_list)
df_consistency = pd.read_csv('data/allthebacteria_accension_range_data.csv')

In [92]:
df_consistency.to_csv('data/allthebacteria_accension_range_data.csv')

In [93]:
size_tf = df_consistency[df_consistency['length']>4].sort_values('length').groupby('consistent').size()

In [94]:
size_tf[True],size_tf[False]

(45564, 21635)

In [45]:
len(df_consistency['length'].unique())

748

In [46]:
max(df_consistency['length'].unique())

5150

In [48]:
df_consistency['length'].mean()

10.434223240779538

In [73]:
length = 5

In [74]:
len(df_consistency[df_consistency['length']>=length]['length'])

67199

In [75]:
df_consistency[df_consistency['length']>=length]['length'].sum()

2205578

In [76]:
df_consistency[df_consistency['length']>=length]['length'][df_consistency['consistent']==True].sum()

1460287

In [78]:
df_consistency[df_consistency['length']>=length]['length'][df_consistency['consistent']==False].sum()

745291

In [77]:
df_consistency[df_consistency['length']>=length].sort_values('length').groupby('consistent').size()

consistent
False    21635
True     45564
dtype: int64